# BPMN Moddle Test Notebook

This notebook demonstrates how to use `bpmn-moddle` from Python in JupyterLite.

The library is loaded via a comm-based bridge to the JupyterLab extension.

## Step 1: Load bpmn-moddle

Run this cell to load the bpmn-moddle library via the extension bridge.

In [ ]:
from bpmn_moddle import load_bpmn_moddle

BpmnModdle = await load_bpmn_moddle()
print(f"bpmn-moddle loaded: {BpmnModdle}")

## Step 2: Parse BPMN XML

Use the BpmnModdle instance to parse BPMN 2.0 XML.

In [ ]:
# Create a new BpmnModdle instance
moddle = BpmnModdle.new()
print(f"BpmnModdle instance created: {moddle}")

In [ ]:
# Sample BPMN XML
bpmn_xml = '''<?xml version="1.0" encoding="UTF-8"?>
<bpmn2:definitions xmlns:bpmn2="http://www.omg.org/spec/BPMN/20100524/MODEL"
                   xmlns:bpmndi="http://www.omg.org/spec/BPMN/20100524/DI"
                   id="sample-definitions"
                   targetNamespace="http://bpmn.io/schema/bpmn">
  <bpmn2:process id="Process_1" name="Sample Process" isExecutable="true">
    <bpmn2:startEvent id="StartEvent_1" name="Start"/>
    <bpmn2:task id="Task_1" name="Do Something"/>
    <bpmn2:endEvent id="EndEvent_1" name="End"/>
    <bpmn2:sequenceFlow id="Flow_1" sourceRef="StartEvent_1" targetRef="Task_1"/>
    <bpmn2:sequenceFlow id="Flow_2" sourceRef="Task_1" targetRef="EndEvent_1"/>
  </bpmn2:process>
</bpmn2:definitions>'''

print("BPMN XML loaded")

In [ ]:
# Parse the BPMN XML
result = await moddle.fromXML(bpmn_xml)
definitions = result.rootElement

print(f"Parsed definitions: {definitions.id}")
print(f"Target namespace: {definitions.targetNamespace}")

In [ ]:
# Access process elements
for element in definitions.rootElements:
    print(f"Root element: {element.id} ({element['$type']})")
    
    if hasattr(element, 'flowElements'):
        for flow_element in element.flowElements:
            name = getattr(flow_element, 'name', '(unnamed)')
            print(f"  - {flow_element['$type']}: {flow_element.id} - {name}")

In [ ]:
# Serialize back to XML
from js import Object
output = await moddle.toXML(definitions, Object.fromEntries([['format', True]]))
print(output.xml)

## Alternative: Python Helper Functions

Use the helper functions from `bpmn_moddle` for a more Pythonic interface.

In [ ]:
from bpmn_moddle import parse_bpmn, to_bpmn_xml

# Parse BPMN XML using the helper
result = await parse_bpmn(bpmn_xml)
definitions = result.rootElement
print(f"Parsed: {definitions.id}")

# Serialize to XML
xml_output = await to_bpmn_xml(definitions)
print(xml_output[:500] + '...')

---

## Bonus: localStorage Access

The bridge also provides access to localStorage from the worker.

### Get/Set localStorage Values

Web Workers cannot access localStorage directly, but the bridge provides this capability.

In [ ]:
from bpmn_moddle import get_bridge

bridge = get_bridge()

# Set a value in localStorage
await bridge.set_localstorage('test_key', 'Hello from Pyodide!')
print("Value set in localStorage")

In [ ]:
# Get the value back
value = await bridge.get_localstorage('test_key')
print(f"Retrieved from localStorage: {value}")

In [ ]:
# List all localStorage keys
keys = await bridge.get_localstorage_keys()
print(f"All localStorage keys: {keys}")

In [ ]:
# Clean up - remove the test key
await bridge.remove_localstorage('test_key')
print("Test key removed from localStorage")

---

## Integration with Operaton

Combine bpmn-moddle with the Operaton API to fetch and analyze process definitions.

In [ ]:
# Example: Fetch process definition XML from Operaton and parse it
# (Uncomment when connected to Operaton)

# from operaton import Operaton
# from bpmn_moddle import parse_bpmn
#
# # Get process definition XML
# process_def = Operaton.get('/process-definition')[0]
# xml_response = Operaton.get(f"/process-definition/{process_def['id']}/xml")
# bpmn_xml = xml_response['bpmn20Xml']
#
# # Parse with bpmn-moddle
# result = await parse_bpmn(bpmn_xml)
# definitions = result.rootElement
#
# # Analyze the process
# for element in definitions.rootElements:
#     if hasattr(element, 'flowElements'):
#         print(f"Process: {element.id}")
#         for flow_elem in element.flowElements:
#             print(f"  {flow_elem.$type}: {flow_elem.id}")